In [1]:
# !pip install -U adapter-transformers

In [1]:
from datasets import load_dataset

# dataset = load_dataset("rotten_tomatoes")
# dataset.num_rows
# dataset = load_dataset("s2orc", streaming=True, split='train')
# print(next(iter(dataset)))

scierc_name = 'nsusemiehl/SciERC'
scierc_dataset = load_dataset(scierc_name)
print(scierc_dataset.num_rows)

# acl_arc_name = 'zapsdcn/citation_intent'
# acl_arc_dataset = load_dataset(acl_arc_name)
# print(acl_arc_dataset.num_rows)



Using custom data configuration nsusemiehl--SciERC-f57c64a52b9c80c0
Reusing dataset json (C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)
100%|██████████| 3/3 [00:00<00:00, 999.04it/s]


{'train': 3219, 'test': 974, 'validation': 455}


Using custom data configuration zapsdcn--citation_intent-0b0f6658161cc990
Reusing dataset json (C:\Users\The Doctor\.cache\huggingface\datasets\json\zapsdcn--citation_intent-0b0f6658161cc990\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)
100%|██████████| 3/3 [00:00<00:00, 999.12it/s]

{'train': 1688, 'test': 139, 'validation': 114}


In [2]:
scierc_dataset['train'][0]

{'text': 'The agreement in question involves number in [[ nouns ]] and << reflexive pronouns >> and is syntactic rather than semantic in nature because grammatical number in English , like grammatical gender in languages such as French , is partly arbitrary .',
 'label': 'CONJUNCTION',
 'metadata': [7, 7, 9, 10]}

In [3]:
# Finding the number of labels
import numpy as np
labels = np.unique(np.array(scierc_dataset['train']['label']))
num_of_labels = labels.size

print(labels)
print(num_of_labels)

['COMPARE' 'CONJUNCTION' 'EVALUATE-FOR' 'FEATURE-OF' 'HYPONYM-OF'
 'PART-OF' 'USED-FOR']
7


In [4]:
# encoding the labels
def encode_labels(dataset):
    for i in range(num_of_labels):
        if dataset['label'] == labels[i]:
            dataset['label'] = i
    return dataset

scierc_dataset = scierc_dataset.map(encode_labels)
scierc_dataset['train'][0]

Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-3fa4decd4606a523.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-4d2e52dbe4cdbad6.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-214c6724dd02783d.arrow


{'text': 'The agreement in question involves number in [[ nouns ]] and << reflexive pronouns >> and is syntactic rather than semantic in nature because grammatical number in English , like grammatical gender in languages such as French , is partly arbitrary .',
 'label': 1,
 'metadata': [7, 7, 9, 10]}

In [5]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
scierc_dataset = scierc_dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
scierc_dataset = scierc_dataset.rename_column("label", 'labels')
# Transform to pytorch tensors and only output the required columns
scierc_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-66a8ac042a6f2d30.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-10b38373a51fb787.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-500b8b5d9ccb8f86.arrow


In [6]:
from transformers import RobertaConfig
from transformers import RobertaAdapterModel

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=num_of_labels,
)
model = RobertaAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaAdapterModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Add new adapter
model.add_adapter("sci_erc")
# Add a matching classification head
model.add_classification_head(
    "sci_erc",
    num_labels=num_of_labels,
    id2label={0:'COMPARE', 1:'CONJUNCTION', 2:'EVALUATE-FOR', 
              3:'FEATURE-OF', 4:'HYPONYM-OF', 5:'PART-OF', 6:'USED-FOR'}
)
# Activate the adapter
model.train_adapter("sci_erc")

In [8]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=50,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    logging_steps=10,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
)

# adding f1 metric
from datasets import load_metric
metric = load_metric('f1')
# print(metric.inputs_description)

def compute_accuracy(EvalPrediction):
  
  logits, labels = EvalPrediction
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels, average= 'macro')

# Adding Support for Tensorboard, supposedly you don't have to do this, but I find that it doesn't work
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback
writer = SummaryWriter()
writer = TensorBoardCallback(writer)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=scierc_dataset["train"],
    eval_dataset=scierc_dataset["validation"],
    compute_metrics=compute_accuracy,    
)

In [ ]:
trainer.train()

In [11]:
trainer.evaluate(scierc_dataset['test'])

***** Running Evaluation *****
  Num examples = 974
  Batch size = 128
100%|██████████| 8/8 [00:02<00:00,  3.23it/s]


{'eval_loss': 0.5086760520935059,
 'eval_f1': 0.7805358759047997,
 'eval_runtime': 2.8486,
 'eval_samples_per_second': 341.922,
 'eval_steps_per_second': 2.808,
 'epoch': 50.0}

In [10]:
trainer.evaluate(scierc_dataset['test'])

***** Running Evaluation *****
  Num examples = 974
  Batch size = 128
100%|██████████| 8/8 [00:02<00:00,  3.78it/s]


{'eval_loss': 0.5021710991859436,
 'eval_f1': 0.8039896126065853,
 'eval_runtime': 2.4362,
 'eval_samples_per_second': 399.801,
 'eval_steps_per_second': 3.284,
 'epoch': 50.0}

In [10]:
trainer.evaluate(scierc_dataset['test'])

***** Running Evaluation *****
  Num examples = 974
  Batch size = 128
100%|██████████| 8/8 [00:02<00:00,  3.28it/s]


{'eval_loss': 0.5224833488464355,
 'eval_f1': 0.779067747226147,
 'eval_runtime': 2.8045,
 'eval_samples_per_second': 347.293,
 'eval_steps_per_second': 2.853,
 'epoch': 50.0}